In [ ]:
%matplotlib inline

In [1]:
from KaggleDataLoader import *

# Create dataframe from files, perform basic cleaning
kaggle_users = create_merged_users_details_file()
kaggle_users.head()

kaggle_taps = create_merged_taps_dataframe()
kaggle_taps = clean_bad_values(kaggle_taps)
kaggle_taps = clean_incompatible_user_ids(kaggle_taps)
kaggle_taps.head()


# Filter outliers
def filter_column_by_quantile(df, column, threshold):
    len_before = len(df)
    df = df[df[column] < np.percentile(df[column], threshold)]
    len_after = len(df)
    print("Filtered out {} rows with outliers in column '{}'".format((len_before - len_after), column))
    return df


def plot_percentiles_of_column(df, col, start, end, bins):
    X = np.linspace(start, end, bins)
    Y = [np.percentile(df[col], x) for x in X]
    plt.plot(X, Y)
    plt.title(col + " Percentiles")
    plt.xlabel("Percent")
    plt.ylabel("Percentile Value")
    plt.show()


# Filter out outliers of HoldTime:
plot_percentiles_of_column(kaggle_taps, 'HoldTime', 99.96, 99.9999, 20)
# After the percentile 99.993 we see significantly higher values, which are definitely outliers.
kaggle_taps = filter_column_by_quantile(kaggle_taps, 'HoldTime', 99.993)

# Add parsed date and time column + calculate cumulative time
kaggle_taps = add_cumulative_timestamps_column(kaggle_taps)


# Group to bin indexes by the cumulative timestamps
def build_bins(df):
    df["PressTimeCumulative"] = df["PressTimeCumulative"] / 1000
    max_press = (int(max((df["PressTimeCumulative"])) / 90) + 1) * 90 + 1
    user_bins = [i for i in range(0, max_press, 90)]
    df["binIndex"] = pd.cut((df["PressTimeCumulative"]), user_bins)
    return df


kaggle_taps = build_bins(kaggle_taps)

# Keep only necessary columns and save to file
kaggle_taps = kaggle_taps[TAPS_FINAL_COLUMNS + ['binIndex']]
kaggle_taps.head()

kaggle_taps.to_csv(constants.KAGGLE_TAPS_INPUT)

Loading taps files: 100/622


Loading taps files: 200/622


Loading taps files: 300/622


KeyboardInterrupt: 